<a href="https://colab.research.google.com/github/figueiredoelizeu/Exercicios_Python_DIO/blob/main/transfer_learning_cat_and_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# 1️⃣ Importar bibliotecas
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# 2️⃣ Carregar o dataset
(train_ds, test_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# 3️⃣ Pré-processamento das imagens
IMG_SIZE = 160

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_ds = train_ds.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)

# 4️⃣ Importar o modelo base MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # congelar as camadas base

# 5️⃣ Criar o modelo final
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)  # saída binária
])

# 6️⃣ Compilar o modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# 7️⃣ Treinar o modelo
EPOCHS = 5
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

# 8️⃣ Avaliar desempenho
loss, acc = model.evaluate(test_ds)
print(f'Precisão final: {acc * 100:.2f}%')

# 9️⃣ Fazer previsões em imagens novas
for image_batch, label_batch in test_ds.take(1):
    prediction = model.predict(image_batch)
    predicted_label = tf.round

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 429s 724ms/step - accuracy: 0.9120 - loss: 0.1837 - val_accuracy: 0.9779 - val_loss: 0.0614
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 429s 737ms/step - accuracy: 0.9785 - loss: 0.0586 - val_accuracy: 0.9796 - val_loss: 0.0573
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 484s 831ms/step - accuracy: 0.9820 - loss: 0.0515 - val_accuracy: 0.9796 - val_loss: 0.0567
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 448s 739ms/step - accuracy: 0.9835 - loss: 0.0473 - val_accuracy: 0.9794 - val_loss: 0.0569
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 417s 717ms/step - accuracy: 0.9848 - loss: 0.0441 - val_accuracy: 0.9791 - val_loss: 0.0575
146/146 ━━━━━━━━━━━━━━━━━━━━ 83s 566ms/step - accuracy: 0.9815 - loss: 0.0506
Precisão final: 97.91%
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
